In [1]:
## This file implements neural networks and logistic regression on p0017Spresabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 99.19% (97.58% after improvements and regularization) for combination data
## and 97.2% for over-sampling data after improvements and regularization.
## For logistic regression, the accuracies are 97.58% for combination data and 95.1% for over-sampling data.
## For random forest, the accuracy is 99.19% for combination and 95.1% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.26% for combination data, and 97.28% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0017Spresabs_quant.csv')
df.shape

(255, 146)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.214333
1      0.350500
2      0.517333
3      0.188833
4      0.191833
5      0.348667
6      0.182667
7      0.207000
8      0.405500
9      0.173500
10     0.184000
11     0.528667
12     0.211667
13     0.218667
14     0.167500
15     0.183333
16     0.406833
17     0.407333
18     0.321167
19     0.177167
20     0.271000
21     0.241833
22     0.178333
23     0.172500
24     0.205833
25     0.170000
26     0.224667
27     0.170500
28     0.163667
29     0.471000
         ...   
225    0.303700
226    0.310600
227    0.443300
228    0.388000
229    0.541200
230    0.378667
231    0.242167
232    0.261500
233    0.295833
234    0.193167
235    0.391333
236    0.160500
237    0.159333
238    0.172000
239    0.262000
240    0.244833
241    0.279833
242    0.200167
243    0.168333
244    0.548667
245    0.164167
246    0.202333
247    0.194167
248    0.188167
249    0.203333
250    0.444667
251    0.238000
252    0.167000
253    0.314333
254    0.252000
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,...,group_2403,group_3458,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,pheno
0,107,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0.214333
1,109,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0.350500
2,115,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0.517333
3,120335,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0.188833
4,120337,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0.191833


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

0    237
1     18
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 146)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 145)

In [13]:
df_clean.head()

,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,...,group_2403,group_3458,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,pheno
0,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 144) (255,)


In [15]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 197), (1, 214)]


Using TensorFlow backend.


In [16]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 237), (1, 237)]


In [17]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [18]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 287 samples, validate on 124 samples
Epoch 1/1000
287/287 [==============================] - 0s 481us/step - loss: 0.7284 - accuracy: 0.6063 - val_loss: 0.6263 - val_accuracy: 0.7581
Epoch 2/1000
287/287 [==============================] - 0s 84us/step - loss: 0.6310 - accuracy: 0.7178 - val_loss: 0.5967 - val_accuracy: 0.7339
Epoch 3/1000
287/287 [==============================] - 0s 86us/step - loss: 0.6092 - accuracy: 0.6794 - val_loss: 0.5805 - val_accuracy: 0.7258
Epoch 4/1000
287/287 [==============================] - 0s 105us/step - loss: 0.5930 - accuracy: 0.6864 - val_loss: 0.5677 - val_accuracy: 0.7581
Epoch 5/1000
287/287 [==============================] - 0s 97us/step - loss: 0.5807 - accuracy: 0.7352 - val_loss: 0.5558 - val_accuracy: 0.7742
Epoch 6/1000
287/287 [==============================] - 0s 101us/step - loss: 0.5682 - accuracy: 0.7596 - val_loss: 0.5448 - val_accuracy: 0.7742
Epoch 7/1000
287/287 [==============================] - 0s 100us/step - loss: 0.5

Epoch 57/1000
287/287 [==============================] - 0s 146us/step - loss: 0.3283 - accuracy: 0.8990 - val_loss: 0.3304 - val_accuracy: 0.9032
Epoch 58/1000
287/287 [==============================] - 0s 135us/step - loss: 0.3259 - accuracy: 0.8990 - val_loss: 0.3286 - val_accuracy: 0.9032
Epoch 59/1000
287/287 [==============================] - 0s 133us/step - loss: 0.3224 - accuracy: 0.9024 - val_loss: 0.3264 - val_accuracy: 0.9032
Epoch 60/1000
287/287 [==============================] - 0s 118us/step - loss: 0.3191 - accuracy: 0.9024 - val_loss: 0.3242 - val_accuracy: 0.9032
Epoch 61/1000
287/287 [==============================] - 0s 81us/step - loss: 0.3164 - accuracy: 0.9024 - val_loss: 0.3206 - val_accuracy: 0.9032
Epoch 62/1000
287/287 [==============================] - 0s 83us/step - loss: 0.3149 - accuracy: 0.9059 - val_loss: 0.3184 - val_accuracy: 0.9032
Epoch 63/1000
287/287 [==============================] - 0s 83us/step - loss: 0.3119 - accuracy: 0.9059 - val_loss: 0.31

Epoch 113/1000
287/287 [==============================] - 0s 80us/step - loss: 0.2162 - accuracy: 0.9199 - val_loss: 0.2381 - val_accuracy: 0.8952
Epoch 114/1000
287/287 [==============================] - 0s 67us/step - loss: 0.2155 - accuracy: 0.9129 - val_loss: 0.2373 - val_accuracy: 0.8952
Epoch 115/1000
287/287 [==============================] - 0s 80us/step - loss: 0.2143 - accuracy: 0.9164 - val_loss: 0.2344 - val_accuracy: 0.9113
Epoch 116/1000
287/287 [==============================] - 0s 79us/step - loss: 0.2124 - accuracy: 0.9164 - val_loss: 0.2344 - val_accuracy: 0.8952
Epoch 117/1000
287/287 [==============================] - 0s 78us/step - loss: 0.2129 - accuracy: 0.9129 - val_loss: 0.2332 - val_accuracy: 0.8952
Epoch 118/1000
287/287 [==============================] - 0s 98us/step - loss: 0.2099 - accuracy: 0.9164 - val_loss: 0.2312 - val_accuracy: 0.9113
Epoch 119/1000
287/287 [==============================] - 0s 81us/step - loss: 0.2085 - accuracy: 0.9233 - val_loss: 0

287/287 [==============================] - 0s 91us/step - loss: 0.1601 - accuracy: 0.9617 - val_loss: 0.1862 - val_accuracy: 0.9435
Epoch 169/1000
287/287 [==============================] - 0s 356us/step - loss: 0.1594 - accuracy: 0.9617 - val_loss: 0.1848 - val_accuracy: 0.9516
Epoch 170/1000
287/287 [==============================] - 0s 127us/step - loss: 0.1598 - accuracy: 0.9547 - val_loss: 0.1840 - val_accuracy: 0.9516
Epoch 171/1000
287/287 [==============================] - 0s 126us/step - loss: 0.1587 - accuracy: 0.9652 - val_loss: 0.1831 - val_accuracy: 0.9516
Epoch 172/1000
287/287 [==============================] - 0s 91us/step - loss: 0.1580 - accuracy: 0.9547 - val_loss: 0.1839 - val_accuracy: 0.9435
Epoch 173/1000
287/287 [==============================] - 0s 111us/step - loss: 0.1558 - accuracy: 0.9617 - val_loss: 0.1817 - val_accuracy: 0.9516
Epoch 174/1000
287/287 [==============================] - 0s 76us/step - loss: 0.1554 - accuracy: 0.9547 - val_loss: 0.1838 - val

287/287 [==============================] - 0s 88us/step - loss: 0.1264 - accuracy: 0.9721 - val_loss: 0.1581 - val_accuracy: 0.9516
Epoch 224/1000
287/287 [==============================] - 0s 77us/step - loss: 0.1262 - accuracy: 0.9686 - val_loss: 0.1581 - val_accuracy: 0.9516
Epoch 225/1000
287/287 [==============================] - 0s 84us/step - loss: 0.1253 - accuracy: 0.9686 - val_loss: 0.1596 - val_accuracy: 0.9758
Epoch 226/1000
287/287 [==============================] - 0s 67us/step - loss: 0.1248 - accuracy: 0.9686 - val_loss: 0.1620 - val_accuracy: 0.9677
Epoch 227/1000
287/287 [==============================] - 0s 62us/step - loss: 0.1241 - accuracy: 0.9756 - val_loss: 0.1555 - val_accuracy: 0.9516
Epoch 228/1000
287/287 [==============================] - 0s 73us/step - loss: 0.1242 - accuracy: 0.9686 - val_loss: 0.1584 - val_accuracy: 0.9758
Epoch 229/1000
287/287 [==============================] - 0s 88us/step - loss: 0.1230 - accuracy: 0.9756 - val_loss: 0.1572 - val_acc

...

Epoch 777/1000
287/287 [==============================] - 0s 117us/step - loss: 0.0354 - accuracy: 1.0000 - val_loss: 0.1068 - val_accuracy: 0.9839
Epoch 778/1000
287/287 [==============================] - 0s 116us/step - loss: 0.0351 - accuracy: 0.9930 - val_loss: 0.1060 - val_accuracy: 0.9919
Epoch 779/1000
287/287 [==============================] - 0s 112us/step - loss: 0.0353 - accuracy: 1.0000 - val_loss: 0.1063 - val_accuracy: 0.9919
Epoch 780/1000
287/287 [==============================] - 0s 107us/step - loss: 0.0349 - accuracy: 0.9965 - val_loss: 0.1056 - val_accuracy: 0.9919
Epoch 781/1000
287/287 [==============================] - 0s 105us/step - loss: 0.0352 - accuracy: 1.0000 - val_loss: 0.1060 - val_accuracy: 0.9919
Epoch 782/1000
287/287 [==============================] - 0s 103us/step - loss: 0.0351 - accuracy: 1.0000 - val_loss: 0.1067 - val_accuracy: 0.9839
Epoch 783/1000
287/287 [==============================] - 0s 100us/step - loss: 0.0351 - accuracy: 0.9930 - val_

287/287 [==============================] - 0s 145us/step - loss: 0.0321 - accuracy: 1.0000 - val_loss: 0.1057 - val_accuracy: 0.9919
Epoch 833/1000
287/287 [==============================] - 0s 152us/step - loss: 0.0325 - accuracy: 1.0000 - val_loss: 0.1064 - val_accuracy: 0.9919
Epoch 834/1000
287/287 [==============================] - 0s 149us/step - loss: 0.0325 - accuracy: 1.0000 - val_loss: 0.1062 - val_accuracy: 0.9919
Epoch 835/1000
287/287 [==============================] - 0s 127us/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 0.1054 - val_accuracy: 0.9919
Epoch 836/1000
287/287 [==============================] - 0s 110us/step - loss: 0.0325 - accuracy: 1.0000 - val_loss: 0.1058 - val_accuracy: 0.9919
Epoch 837/1000
287/287 [==============================] - 0s 108us/step - loss: 0.0323 - accuracy: 1.0000 - val_loss: 0.1057 - val_accuracy: 0.9919
Epoch 838/1000
287/287 [==============================] - 0s 131us/step - loss: 0.0319 - accuracy: 1.0000 - val_loss: 0.1056 - 

287/287 [==============================] - 0s 143us/step - loss: 0.0297 - accuracy: 1.0000 - val_loss: 0.1060 - val_accuracy: 0.9919
Epoch 888/1000
287/287 [==============================] - 0s 126us/step - loss: 0.0296 - accuracy: 1.0000 - val_loss: 0.1059 - val_accuracy: 0.9919
Epoch 889/1000
287/287 [==============================] - 0s 109us/step - loss: 0.0294 - accuracy: 1.0000 - val_loss: 0.1061 - val_accuracy: 0.9919
Epoch 890/1000
287/287 [==============================] - 0s 112us/step - loss: 0.0293 - accuracy: 1.0000 - val_loss: 0.1063 - val_accuracy: 0.9919
Epoch 891/1000
287/287 [==============================] - 0s 119us/step - loss: 0.0292 - accuracy: 1.0000 - val_loss: 0.1061 - val_accuracy: 0.9919
Epoch 892/1000
287/287 [==============================] - 0s 148us/step - loss: 0.0294 - accuracy: 1.0000 - val_loss: 0.1072 - val_accuracy: 0.9919
Epoch 893/1000
287/287 [==============================] - 0s 130us/step - loss: 0.0292 - accuracy: 1.0000 - val_loss: 0.1065 - 

287/287 [==============================] - 0s 120us/step - loss: 0.0271 - accuracy: 1.0000 - val_loss: 0.1062 - val_accuracy: 0.9919
Epoch 943/1000
287/287 [==============================] - 0s 86us/step - loss: 0.0272 - accuracy: 1.0000 - val_loss: 0.1062 - val_accuracy: 0.9919
Epoch 944/1000
287/287 [==============================] - 0s 78us/step - loss: 0.0269 - accuracy: 1.0000 - val_loss: 0.1065 - val_accuracy: 0.9919
Epoch 945/1000
287/287 [==============================] - 0s 79us/step - loss: 0.0270 - accuracy: 1.0000 - val_loss: 0.1065 - val_accuracy: 0.9919
Epoch 946/1000
287/287 [==============================] - 0s 136us/step - loss: 0.0278 - accuracy: 1.0000 - val_loss: 0.1063 - val_accuracy: 0.9919
Epoch 947/1000
287/287 [==============================] - 0s 183us/step - loss: 0.0273 - accuracy: 1.0000 - val_loss: 0.1067 - val_accuracy: 0.9919
Epoch 948/1000
287/287 [==============================] - 0s 152us/step - loss: 0.0269 - accuracy: 1.0000 - val_loss: 0.1066 - val

287/287 [==============================] - 0s 147us/step - loss: 0.0250 - accuracy: 1.0000 - val_loss: 0.1069 - val_accuracy: 0.9919
Epoch 998/1000
287/287 [==============================] - 0s 121us/step - loss: 0.0249 - accuracy: 1.0000 - val_loss: 0.1071 - val_accuracy: 0.9919
Epoch 999/1000
287/287 [==============================] - 0s 124us/step - loss: 0.0251 - accuracy: 1.0000 - val_loss: 0.1071 - val_accuracy: 0.9919
Epoch 1000/1000
287/287 [==============================] - 0s 252us/step - loss: 0.0249 - accuracy: 1.0000 - val_loss: 0.1069 - val_accuracy: 0.9919


In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

124/124 [==============================] - 0s 84us/step
combination test accuracy: 99.19%


In [25]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [26]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 287 samples, validate on 124 samples
Epoch 1/1000
287/287 [==============================] - 0s 744us/step - loss: 2.3897 - accuracy: 0.6655 - val_loss: 0.9999 - val_accuracy: 0.5887
Epoch 2/1000
287/287 [==============================] - 0s 88us/step - loss: 2.2390 - accuracy: 0.6620 - val_loss: 0.9474 - val_accuracy: 0.6532
Epoch 3/1000
287/287 [==============================] - 0s 134us/step - loss: 2.1856 - accuracy: 0.6829 - val_loss: 0.9010 - val_accuracy: 0.6613
Epoch 4/1000
287/287 [==============================] - 0s 156us/step - loss: 2.2477 - accuracy: 0.6864 - val_loss: 0.8621 - val_accuracy: 0.6613
Epoch 5/1000
287/287 [==============================] - 0s 163us/step - loss: 2.1191 - accuracy: 0.6760 - val_loss: 0.8227 - val_accuracy: 0.6613
Epoch 6/1000
287/287 [==============================] - 0s 86us/step - loss: 2.5760 - accuracy: 0.6760 - val_loss: 0.7965 - val_accuracy: 0.6613
Epoch 7/1000
287/287 [==============================] - 0s 101us/step - loss: 2.

Epoch 57/1000
287/287 [==============================] - 0s 178us/step - loss: 1.8112 - accuracy: 0.8223 - val_loss: 0.4936 - val_accuracy: 0.9032
Epoch 58/1000
287/287 [==============================] - 0s 142us/step - loss: 1.9168 - accuracy: 0.8153 - val_loss: 0.4917 - val_accuracy: 0.9032
Epoch 59/1000
287/287 [==============================] - 0s 95us/step - loss: 1.6859 - accuracy: 0.8328 - val_loss: 0.4893 - val_accuracy: 0.9032
Epoch 60/1000
287/287 [==============================] - 0s 97us/step - loss: 1.9134 - accuracy: 0.8049 - val_loss: 0.4898 - val_accuracy: 0.8952
Epoch 61/1000
287/287 [==============================] - 0s 101us/step - loss: 2.2811 - accuracy: 0.7875 - val_loss: 0.4876 - val_accuracy: 0.8952
Epoch 62/1000
287/287 [==============================] - 0s 115us/step - loss: 2.0717 - accuracy: 0.8049 - val_loss: 0.4844 - val_accuracy: 0.8952
Epoch 63/1000
287/287 [==============================] - 0s 80us/step - loss: 1.8899 - accuracy: 0.8258 - val_loss: 0.48

Epoch 113/1000
287/287 [==============================] - 0s 68us/step - loss: 1.9177 - accuracy: 0.8432 - val_loss: 0.3947 - val_accuracy: 0.9274
Epoch 114/1000
287/287 [==============================] - 0s 65us/step - loss: 1.7196 - accuracy: 0.8223 - val_loss: 0.3926 - val_accuracy: 0.9274
Epoch 115/1000
287/287 [==============================] - 0s 65us/step - loss: 2.1323 - accuracy: 0.8188 - val_loss: 0.3906 - val_accuracy: 0.9274
Epoch 116/1000
287/287 [==============================] - 0s 73us/step - loss: 1.9101 - accuracy: 0.8467 - val_loss: 0.3909 - val_accuracy: 0.9274
Epoch 117/1000
287/287 [==============================] - 0s 74us/step - loss: 1.9644 - accuracy: 0.8328 - val_loss: 0.3895 - val_accuracy: 0.9274
Epoch 118/1000
287/287 [==============================] - 0s 72us/step - loss: 1.8634 - accuracy: 0.8432 - val_loss: 0.3866 - val_accuracy: 0.9274
Epoch 119/1000
287/287 [==============================] - 0s 76us/step - loss: 2.1997 - accuracy: 0.8084 - val_loss: 0

287/287 [==============================] - 0s 80us/step - loss: 1.6562 - accuracy: 0.8571 - val_loss: 0.3295 - val_accuracy: 0.9435
Epoch 169/1000
287/287 [==============================] - 0s 81us/step - loss: 1.8677 - accuracy: 0.8502 - val_loss: 0.3289 - val_accuracy: 0.9435
Epoch 170/1000
287/287 [==============================] - 0s 68us/step - loss: 1.4925 - accuracy: 0.8711 - val_loss: 0.3281 - val_accuracy: 0.9435
Epoch 171/1000
287/287 [==============================] - 0s 70us/step - loss: 1.8174 - accuracy: 0.8537 - val_loss: 0.3271 - val_accuracy: 0.9435
Epoch 172/1000
287/287 [==============================] - 0s 77us/step - loss: 1.6961 - accuracy: 0.8746 - val_loss: 0.3265 - val_accuracy: 0.9355
Epoch 173/1000
287/287 [==============================] - 0s 82us/step - loss: 1.8096 - accuracy: 0.8606 - val_loss: 0.3250 - val_accuracy: 0.9435
Epoch 174/1000
287/287 [==============================] - 0s 70us/step - loss: 1.5902 - accuracy: 0.8780 - val_loss: 0.3244 - val_acc

Epoch 224/1000
287/287 [==============================] - 0s 69us/step - loss: 1.6253 - accuracy: 0.8641 - val_loss: 0.2942 - val_accuracy: 0.9597
Epoch 225/1000
287/287 [==============================] - 0s 66us/step - loss: 1.4593 - accuracy: 0.8955 - val_loss: 0.2944 - val_accuracy: 0.9435
Epoch 226/1000
287/287 [==============================] - 0s 63us/step - loss: 1.7416 - accuracy: 0.8571 - val_loss: 0.2934 - val_accuracy: 0.9597
Epoch 227/1000
287/287 [==============================] - 0s 63us/step - loss: 1.8882 - accuracy: 0.8571 - val_loss: 0.2930 - val_accuracy: 0.9597
Epoch 228/1000
287/287 [==============================] - 0s 73us/step - loss: 1.6223 - accuracy: 0.8850 - val_loss: 0.2928 - val_accuracy: 0.9597
Epoch 229/1000
287/287 [==============================] - 0s 66us/step - loss: 1.6269 - accuracy: 0.8711 - val_loss: 0.2918 - val_accuracy: 0.9597
Epoch 230/1000
287/287 [==============================] - 0s 73us/step - loss: 2.3232 - accuracy: 0.8223 - val_loss: 0

...

Epoch 728/1000
287/287 [==============================] - 0s 75us/step - loss: 1.4758 - accuracy: 0.9094 - val_loss: 0.2104 - val_accuracy: 0.9919
Epoch 729/1000
287/287 [==============================] - 0s 71us/step - loss: 1.7959 - accuracy: 0.8885 - val_loss: 0.2106 - val_accuracy: 0.9919
Epoch 730/1000
287/287 [==============================] - 0s 73us/step - loss: 1.6896 - accuracy: 0.8955 - val_loss: 0.2106 - val_accuracy: 0.9919
Epoch 731/1000
287/287 [==============================] - 0s 70us/step - loss: 2.0113 - accuracy: 0.8746 - val_loss: 0.2108 - val_accuracy: 0.9919
Epoch 732/1000
287/287 [==============================] - 0s 72us/step - loss: 2.3892 - accuracy: 0.8502 - val_loss: 0.2105 - val_accuracy: 0.9919
Epoch 733/1000
287/287 [==============================] - 0s 71us/step - loss: 2.1146 - accuracy: 0.8676 - val_loss: 0.2099 - val_accuracy: 0.9919
Epoch 734/1000
287/287 [==============================] - 0s 75us/step - loss: 1.3677 - accuracy: 0.9164 - val_loss: 0

Epoch 784/1000
287/287 [==============================] - 0s 73us/step - loss: 1.9563 - accuracy: 0.8780 - val_loss: 0.2087 - val_accuracy: 0.9919
Epoch 785/1000
287/287 [==============================] - 0s 79us/step - loss: 1.6818 - accuracy: 0.8955 - val_loss: 0.2087 - val_accuracy: 0.9919
Epoch 786/1000
287/287 [==============================] - 0s 71us/step - loss: 1.9481 - accuracy: 0.8780 - val_loss: 0.2085 - val_accuracy: 0.9919
Epoch 787/1000
287/287 [==============================] - 0s 70us/step - loss: 1.7943 - accuracy: 0.8885 - val_loss: 0.2099 - val_accuracy: 0.9758
Epoch 788/1000
287/287 [==============================] - 0s 71us/step - loss: 1.5761 - accuracy: 0.9024 - val_loss: 0.2086 - val_accuracy: 0.9919
Epoch 789/1000
287/287 [==============================] - 0s 72us/step - loss: 1.6303 - accuracy: 0.8990 - val_loss: 0.2083 - val_accuracy: 0.9919
Epoch 790/1000
287/287 [==============================] - 0s 71us/step - loss: 2.0113 - accuracy: 0.8746 - val_loss: 0

Epoch 840/1000
287/287 [==============================] - 0s 73us/step - loss: 1.8991 - accuracy: 0.8815 - val_loss: 0.2066 - val_accuracy: 0.9758
Epoch 841/1000
287/287 [==============================] - 0s 69us/step - loss: 1.6825 - accuracy: 0.8955 - val_loss: 0.2054 - val_accuracy: 0.9919
Epoch 842/1000
287/287 [==============================] - 0s 71us/step - loss: 1.9527 - accuracy: 0.8780 - val_loss: 0.2073 - val_accuracy: 0.9758
Epoch 843/1000
287/287 [==============================] - 0s 71us/step - loss: 1.2515 - accuracy: 0.9233 - val_loss: 0.2070 - val_accuracy: 0.9758
Epoch 844/1000
287/287 [==============================] - 0s 71us/step - loss: 2.1689 - accuracy: 0.8641 - val_loss: 0.2063 - val_accuracy: 0.9758
Epoch 845/1000
287/287 [==============================] - 0s 70us/step - loss: 1.5234 - accuracy: 0.9059 - val_loss: 0.2055 - val_accuracy: 0.9919
Epoch 846/1000
287/287 [==============================] - 0s 70us/step - loss: 1.7345 - accuracy: 0.8920 - val_loss: 0

Epoch 896/1000
287/287 [==============================] - 0s 75us/step - loss: 1.9474 - accuracy: 0.8780 - val_loss: 0.2043 - val_accuracy: 0.9758
Epoch 897/1000
287/287 [==============================] - 0s 70us/step - loss: 2.1114 - accuracy: 0.8676 - val_loss: 0.2055 - val_accuracy: 0.9758
Epoch 898/1000
287/287 [==============================] - 0s 69us/step - loss: 1.7343 - accuracy: 0.8920 - val_loss: 0.2033 - val_accuracy: 0.9919
Epoch 899/1000
287/287 [==============================] - 0s 71us/step - loss: 1.6825 - accuracy: 0.8955 - val_loss: 0.2046 - val_accuracy: 0.9758
Epoch 900/1000
287/287 [==============================] - 0s 73us/step - loss: 1.9458 - accuracy: 0.8780 - val_loss: 0.2051 - val_accuracy: 0.9758
Epoch 901/1000
287/287 [==============================] - 0s 69us/step - loss: 1.6809 - accuracy: 0.8955 - val_loss: 0.2034 - val_accuracy: 0.9919
Epoch 902/1000
287/287 [==============================] - 0s 81us/step - loss: 2.0560 - accuracy: 0.8711 - val_loss: 0

Epoch 952/1000
287/287 [==============================] - 0s 75us/step - loss: 2.0005 - accuracy: 0.8746 - val_loss: 0.2026 - val_accuracy: 0.9758
Epoch 953/1000
287/287 [==============================] - 0s 70us/step - loss: 1.8375 - accuracy: 0.8850 - val_loss: 0.2014 - val_accuracy: 0.9919
Epoch 954/1000
287/287 [==============================] - 0s 69us/step - loss: 1.7835 - accuracy: 0.8885 - val_loss: 0.2018 - val_accuracy: 0.9758
Epoch 955/1000
287/287 [==============================] - 0s 72us/step - loss: 1.4099 - accuracy: 0.9129 - val_loss: 0.2022 - val_accuracy: 0.9758
Epoch 956/1000
287/287 [==============================] - 0s 71us/step - loss: 1.6795 - accuracy: 0.8955 - val_loss: 0.2028 - val_accuracy: 0.9758
Epoch 957/1000
287/287 [==============================] - 0s 67us/step - loss: 1.6731 - accuracy: 0.8955 - val_loss: 0.2028 - val_accuracy: 0.9758
Epoch 958/1000
287/287 [==============================] - 0s 71us/step - loss: 1.7313 - accuracy: 0.8920 - val_loss: 0

In [28]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

124/124 [==============================] - 0s 69us/step
combination test accuracy: 97.58%


In [29]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [30]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 315us/step - loss: 0.7418 - accuracy: 0.4350 - val_loss: 0.6895 - val_accuracy: 0.6294
Epoch 2/1000
331/331 [==============================] - 0s 64us/step - loss: 0.6750 - accuracy: 0.6435 - val_loss: 0.6530 - val_accuracy: 0.6783
Epoch 3/1000
331/331 [==============================] - 0s 61us/step - loss: 0.6436 - accuracy: 0.6858 - val_loss: 0.6235 - val_accuracy: 0.6993
Epoch 4/1000
331/331 [==============================] - 0s 67us/step - loss: 0.6180 - accuracy: 0.6949 - val_loss: 0.6004 - val_accuracy: 0.7273
Epoch 5/1000
331/331 [==============================] - 0s 67us/step - loss: 0.5978 - accuracy: 0.7281 - val_loss: 0.5828 - val_accuracy: 0.7343
Epoch 6/1000
331/331 [==============================] - 0s 77us/step - loss: 0.5794 - accuracy: 0.7432 - val_loss: 0.5656 - val_accuracy: 0.7343
Epoch 7/1000
331/331 [==============================] - 0s 70us/step - loss: 0.5645

331/331 [==============================] - 0s 61us/step - loss: 0.3287 - accuracy: 0.9154 - val_loss: 0.3391 - val_accuracy: 0.8601
Epoch 57/1000
331/331 [==============================] - 0s 59us/step - loss: 0.3257 - accuracy: 0.9184 - val_loss: 0.3377 - val_accuracy: 0.8601
Epoch 58/1000
331/331 [==============================] - 0s 59us/step - loss: 0.3242 - accuracy: 0.9154 - val_loss: 0.3354 - val_accuracy: 0.8601
Epoch 59/1000
331/331 [==============================] - 0s 61us/step - loss: 0.3220 - accuracy: 0.9215 - val_loss: 0.3348 - val_accuracy: 0.8601
Epoch 60/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3188 - accuracy: 0.9184 - val_loss: 0.3339 - val_accuracy: 0.8531
Epoch 61/1000
331/331 [==============================] - 0s 61us/step - loss: 0.3164 - accuracy: 0.9154 - val_loss: 0.3297 - val_accuracy: 0.8601
Epoch 62/1000
331/331 [==============================] - 0s 62us/step - loss: 0.3156 - accuracy: 0.9154 - val_loss: 0.3278 - val_accuracy:

331/331 [==============================] - 0s 69us/step - loss: 0.2370 - accuracy: 0.9366 - val_loss: 0.2606 - val_accuracy: 0.9161
Epoch 113/1000
331/331 [==============================] - 0s 64us/step - loss: 0.2360 - accuracy: 0.9456 - val_loss: 0.2633 - val_accuracy: 0.8881
Epoch 114/1000
331/331 [==============================] - 0s 69us/step - loss: 0.2353 - accuracy: 0.9366 - val_loss: 0.2625 - val_accuracy: 0.8881
Epoch 115/1000
331/331 [==============================] - 0s 61us/step - loss: 0.2348 - accuracy: 0.9426 - val_loss: 0.2691 - val_accuracy: 0.8881
Epoch 116/1000
331/331 [==============================] - 0s 63us/step - loss: 0.2335 - accuracy: 0.9366 - val_loss: 0.2573 - val_accuracy: 0.8951
Epoch 117/1000
331/331 [==============================] - 0s 61us/step - loss: 0.2334 - accuracy: 0.9366 - val_loss: 0.2599 - val_accuracy: 0.8881
Epoch 118/1000
331/331 [==============================] - 0s 63us/step - loss: 0.2315 - accuracy: 0.9366 - val_loss: 0.2595 - val_acc

Epoch 168/1000
331/331 [==============================] - 0s 73us/step - loss: 0.1896 - accuracy: 0.9517 - val_loss: 0.2207 - val_accuracy: 0.9021
Epoch 169/1000
331/331 [==============================] - 0s 63us/step - loss: 0.1888 - accuracy: 0.9456 - val_loss: 0.2256 - val_accuracy: 0.9161
Epoch 170/1000
331/331 [==============================] - 0s 64us/step - loss: 0.1883 - accuracy: 0.9456 - val_loss: 0.2188 - val_accuracy: 0.9021
Epoch 171/1000
331/331 [==============================] - 0s 62us/step - loss: 0.1877 - accuracy: 0.9486 - val_loss: 0.2221 - val_accuracy: 0.9301
Epoch 172/1000
331/331 [==============================] - 0s 64us/step - loss: 0.1868 - accuracy: 0.9456 - val_loss: 0.2212 - val_accuracy: 0.9301
Epoch 173/1000
331/331 [==============================] - 0s 67us/step - loss: 0.1865 - accuracy: 0.9607 - val_loss: 0.2181 - val_accuracy: 0.9021
Epoch 174/1000
331/331 [==============================] - 0s 70us/step - loss: 0.1854 - accuracy: 0.9486 - val_loss: 0

Epoch 224/1000
331/331 [==============================] - 0s 67us/step - loss: 0.1587 - accuracy: 0.9637 - val_loss: 0.2011 - val_accuracy: 0.9231
Epoch 225/1000
331/331 [==============================] - 0s 62us/step - loss: 0.1568 - accuracy: 0.9637 - val_loss: 0.1949 - val_accuracy: 0.9301
Epoch 226/1000
331/331 [==============================] - 0s 60us/step - loss: 0.1576 - accuracy: 0.9728 - val_loss: 0.1915 - val_accuracy: 0.9301
Epoch 227/1000
331/331 [==============================] - 0s 63us/step - loss: 0.1562 - accuracy: 0.9668 - val_loss: 0.1901 - val_accuracy: 0.9231
Epoch 228/1000
331/331 [==============================] - 0s 64us/step - loss: 0.1580 - accuracy: 0.9607 - val_loss: 0.1918 - val_accuracy: 0.9301
Epoch 229/1000
331/331 [==============================] - 0s 60us/step - loss: 0.1544 - accuracy: 0.9728 - val_loss: 0.1961 - val_accuracy: 0.9301
Epoch 230/1000
331/331 [==============================] - 0s 62us/step - loss: 0.1545 - accuracy: 0.9698 - val_loss: 0

...

Epoch 728/1000
331/331 [==============================] - 0s 68us/step - loss: 0.0533 - accuracy: 0.9909 - val_loss: 0.1177 - val_accuracy: 0.9650
Epoch 729/1000
331/331 [==============================] - 0s 68us/step - loss: 0.0519 - accuracy: 0.9940 - val_loss: 0.1174 - val_accuracy: 0.9650
Epoch 730/1000
331/331 [==============================] - 0s 71us/step - loss: 0.0525 - accuracy: 0.9909 - val_loss: 0.1182 - val_accuracy: 0.9650
Epoch 731/1000
331/331 [==============================] - 0s 71us/step - loss: 0.0517 - accuracy: 0.9909 - val_loss: 0.1158 - val_accuracy: 0.9650
Epoch 732/1000
331/331 [==============================] - 0s 73us/step - loss: 0.0522 - accuracy: 0.9940 - val_loss: 0.1157 - val_accuracy: 0.9720
Epoch 733/1000
331/331 [==============================] - 0s 65us/step - loss: 0.0520 - accuracy: 0.9940 - val_loss: 0.1156 - val_accuracy: 0.9720
Epoch 734/1000
331/331 [==============================] - 0s 78us/step - loss: 0.0518 - accuracy: 0.9940 - val_loss: 0

Epoch 784/1000
331/331 [==============================] - 0s 67us/step - loss: 0.0475 - accuracy: 0.9940 - val_loss: 0.1192 - val_accuracy: 0.9650
Epoch 785/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0480 - accuracy: 0.9940 - val_loss: 0.1176 - val_accuracy: 0.9720
Epoch 786/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0483 - accuracy: 0.9940 - val_loss: 0.1153 - val_accuracy: 0.9720
Epoch 787/1000
331/331 [==============================] - 0s 67us/step - loss: 0.0473 - accuracy: 0.9940 - val_loss: 0.1168 - val_accuracy: 0.9720
Epoch 788/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0476 - accuracy: 0.9940 - val_loss: 0.1129 - val_accuracy: 0.9720
Epoch 789/1000
331/331 [==============================] - 0s 61us/step - loss: 0.0475 - accuracy: 0.9940 - val_loss: 0.1171 - val_accuracy: 0.9720
Epoch 790/1000
331/331 [==============================] - 0s 65us/step - loss: 0.0473 - accuracy: 0.9940 - val_loss: 0

Epoch 840/1000
331/331 [==============================] - 0s 67us/step - loss: 0.0437 - accuracy: 0.9940 - val_loss: 0.1126 - val_accuracy: 0.9720
Epoch 841/1000
331/331 [==============================] - 0s 61us/step - loss: 0.0441 - accuracy: 0.9940 - val_loss: 0.1113 - val_accuracy: 0.9720
Epoch 842/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0441 - accuracy: 0.9940 - val_loss: 0.1121 - val_accuracy: 0.9720
Epoch 843/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0437 - accuracy: 0.9940 - val_loss: 0.1125 - val_accuracy: 0.9720
Epoch 844/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0433 - accuracy: 0.9940 - val_loss: 0.1172 - val_accuracy: 0.9720
Epoch 845/1000
331/331 [==============================] - 0s 63us/step - loss: 0.0435 - accuracy: 0.9940 - val_loss: 0.1130 - val_accuracy: 0.9720
Epoch 846/1000
331/331 [==============================] - 0s 62us/step - loss: 0.0430 - accuracy: 0.9940 - val_loss: 0

Epoch 896/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0405 - accuracy: 0.9940 - val_loss: 0.1097 - val_accuracy: 0.9720
Epoch 897/1000
331/331 [==============================] - 0s 63us/step - loss: 0.0400 - accuracy: 0.9940 - val_loss: 0.1148 - val_accuracy: 0.9720
Epoch 898/1000
331/331 [==============================] - 0s 63us/step - loss: 0.0408 - accuracy: 0.9940 - val_loss: 0.1121 - val_accuracy: 0.9720
Epoch 899/1000
331/331 [==============================] - 0s 66us/step - loss: 0.0401 - accuracy: 0.9940 - val_loss: 0.1134 - val_accuracy: 0.9720
Epoch 900/1000
331/331 [==============================] - 0s 65us/step - loss: 0.0404 - accuracy: 0.9940 - val_loss: 0.1137 - val_accuracy: 0.9720
Epoch 901/1000
331/331 [==============================] - 0s 61us/step - loss: 0.0401 - accuracy: 0.9940 - val_loss: 0.1156 - val_accuracy: 0.9720
Epoch 902/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0402 - accuracy: 0.9940 - val_loss: 0

Epoch 952/1000
331/331 [==============================] - 0s 65us/step - loss: 0.0372 - accuracy: 0.9940 - val_loss: 0.1158 - val_accuracy: 0.9720
Epoch 953/1000
331/331 [==============================] - 0s 58us/step - loss: 0.0378 - accuracy: 0.9940 - val_loss: 0.1139 - val_accuracy: 0.9720
Epoch 954/1000
331/331 [==============================] - 0s 62us/step - loss: 0.0373 - accuracy: 0.9940 - val_loss: 0.1113 - val_accuracy: 0.9720
Epoch 955/1000
331/331 [==============================] - 0s 74us/step - loss: 0.0373 - accuracy: 0.9940 - val_loss: 0.1142 - val_accuracy: 0.9720
Epoch 956/1000
331/331 [==============================] - 0s 60us/step - loss: 0.0370 - accuracy: 0.9940 - val_loss: 0.1112 - val_accuracy: 0.9720
Epoch 957/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0369 - accuracy: 0.9940 - val_loss: 0.1121 - val_accuracy: 0.9720
Epoch 958/1000
331/331 [==============================] - 0s 64us/step - loss: 0.0372 - accuracy: 0.9940 - val_loss: 0

In [32]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

143/143 [==============================] - 0s 72us/step
over-sampling test accuracy: 97.20%


In [33]:
#### add regularizer and dropout on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [34]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 474us/step - loss: 2.3512 - accuracy: 0.4653 - val_loss: 0.9877 - val_accuracy: 0.4965
Epoch 2/1000
331/331 [==============================] - 0s 65us/step - loss: 2.8213 - accuracy: 0.4955 - val_loss: 0.9324 - val_accuracy: 0.5245
Epoch 3/1000
331/331 [==============================] - 0s 75us/step - loss: 2.1929 - accuracy: 0.5861 - val_loss: 0.8645 - val_accuracy: 0.5245
Epoch 4/1000
331/331 [==============================] - 0s 77us/step - loss: 2.3850 - accuracy: 0.6133 - val_loss: 0.8268 - val_accuracy: 0.5594
Epoch 5/1000
331/331 [==============================] - 0s 69us/step - loss: 2.3094 - accuracy: 0.6133 - val_loss: 0.7957 - val_accuracy: 0.6154
Epoch 6/1000
331/331 [==============================] - 0s 76us/step - loss: 1.8363 - accuracy: 0.6495 - val_loss: 0.7681 - val_accuracy: 0.6294
Epoch 7/1000
331/331 [==============================] - 0s 69us/step - loss: 2.4578

Epoch 57/1000
331/331 [==============================] - 0s 68us/step - loss: 1.8560 - accuracy: 0.8187 - val_loss: 0.5441 - val_accuracy: 0.7413
Epoch 58/1000
331/331 [==============================] - 0s 66us/step - loss: 1.8953 - accuracy: 0.8036 - val_loss: 0.5428 - val_accuracy: 0.7203
Epoch 59/1000
331/331 [==============================] - 0s 72us/step - loss: 1.9615 - accuracy: 0.8006 - val_loss: 0.5423 - val_accuracy: 0.7972
Epoch 60/1000
331/331 [==============================] - 0s 73us/step - loss: 1.8088 - accuracy: 0.7976 - val_loss: 0.5399 - val_accuracy: 0.7972
Epoch 61/1000
331/331 [==============================] - 0s 65us/step - loss: 1.9602 - accuracy: 0.8036 - val_loss: 0.5370 - val_accuracy: 0.8042
Epoch 62/1000
331/331 [==============================] - 0s 70us/step - loss: 1.8857 - accuracy: 0.8066 - val_loss: 0.5358 - val_accuracy: 0.7622
Epoch 63/1000
331/331 [==============================] - 0s 70us/step - loss: 2.0967 - accuracy: 0.7976 - val_loss: 0.5357 -

331/331 [==============================] - 0s 77us/step - loss: 2.1642 - accuracy: 0.7976 - val_loss: 0.4545 - val_accuracy: 0.8881
Epoch 114/1000
331/331 [==============================] - 0s 74us/step - loss: 1.7363 - accuracy: 0.8308 - val_loss: 0.4565 - val_accuracy: 0.8811
Epoch 115/1000
331/331 [==============================] - 0s 71us/step - loss: 1.8636 - accuracy: 0.8218 - val_loss: 0.4546 - val_accuracy: 0.8601
Epoch 116/1000
331/331 [==============================] - 0s 71us/step - loss: 1.7864 - accuracy: 0.8338 - val_loss: 0.4676 - val_accuracy: 0.8182
Epoch 117/1000
331/331 [==============================] - 0s 70us/step - loss: 2.0176 - accuracy: 0.8218 - val_loss: 0.4510 - val_accuracy: 0.8811
Epoch 118/1000
331/331 [==============================] - 0s 74us/step - loss: 1.8695 - accuracy: 0.8248 - val_loss: 0.4517 - val_accuracy: 0.8811
Epoch 119/1000
331/331 [==============================] - 0s 70us/step - loss: 1.9590 - accuracy: 0.8218 - val_loss: 0.4462 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 106us/step - loss: 1.8168 - accuracy: 0.8610 - val_loss: 0.4014 - val_accuracy: 0.9091
Epoch 170/1000
331/331 [==============================] - 0s 69us/step - loss: 1.5400 - accuracy: 0.8731 - val_loss: 0.3961 - val_accuracy: 0.9091
Epoch 171/1000
331/331 [==============================] - 0s 75us/step - loss: 1.5805 - accuracy: 0.8731 - val_loss: 0.3943 - val_accuracy: 0.8671
Epoch 172/1000
331/331 [==============================] - 0s 72us/step - loss: 1.7235 - accuracy: 0.8640 - val_loss: 0.3991 - val_accuracy: 0.9091
Epoch 173/1000
331/331 [==============================] - 0s 74us/step - loss: 1.3598 - accuracy: 0.8792 - val_loss: 0.3951 - val_accuracy: 0.9091
Epoch 174/1000
331/331 [==============================] - 0s 70us/step - loss: 1.5857 - accuracy: 0.8701 - val_loss: 0.3926 - val_accuracy: 0.8741
Epoch 175/1000
331/331 [==============================] - 0s 67us/step - loss: 1.9599 - accuracy: 0.8489 - val_loss: 

331/331 [==============================] - 0s 72us/step - loss: 1.5508 - accuracy: 0.8701 - val_loss: 0.3577 - val_accuracy: 0.8811
Epoch 225/1000
331/331 [==============================] - 0s 67us/step - loss: 1.7385 - accuracy: 0.8640 - val_loss: 0.3641 - val_accuracy: 0.9091
Epoch 226/1000
331/331 [==============================] - 0s 69us/step - loss: 1.4495 - accuracy: 0.8822 - val_loss: 0.3551 - val_accuracy: 0.9021
Epoch 227/1000
331/331 [==============================] - 0s 66us/step - loss: 1.8808 - accuracy: 0.8550 - val_loss: 0.3545 - val_accuracy: 0.9021
Epoch 228/1000
331/331 [==============================] - 0s 70us/step - loss: 1.6416 - accuracy: 0.8640 - val_loss: 0.3540 - val_accuracy: 0.9021
Epoch 229/1000
331/331 [==============================] - 0s 71us/step - loss: 1.5879 - accuracy: 0.8792 - val_loss: 0.3519 - val_accuracy: 0.8811
Epoch 230/1000
331/331 [==============================] - 0s 79us/step - loss: 1.6849 - accuracy: 0.8640 - val_loss: 0.3548 - val_acc

...

Epoch 727/1000
331/331 [==============================] - 0s 98us/step - loss: 1.7763 - accuracy: 0.8792 - val_loss: 0.2333 - val_accuracy: 0.9720
Epoch 728/1000
331/331 [==============================] - 0s 81us/step - loss: 1.4418 - accuracy: 0.9063 - val_loss: 0.2303 - val_accuracy: 0.9720
Epoch 729/1000
331/331 [==============================] - 0s 85us/step - loss: 1.8194 - accuracy: 0.8761 - val_loss: 0.2275 - val_accuracy: 0.9720
Epoch 730/1000
331/331 [==============================] - 0s 76us/step - loss: 1.8201 - accuracy: 0.8761 - val_loss: 0.2274 - val_accuracy: 0.9720
Epoch 731/1000
331/331 [==============================] - 0s 86us/step - loss: 1.7239 - accuracy: 0.8822 - val_loss: 0.2286 - val_accuracy: 0.9720
Epoch 732/1000
331/331 [==============================] - 0s 78us/step - loss: 1.6813 - accuracy: 0.8882 - val_loss: 0.2278 - val_accuracy: 0.9720
Epoch 733/1000
331/331 [==============================] - 0s 70us/step - loss: 1.3955 - accuracy: 0.9094 - val_loss: 0

Epoch 783/1000
331/331 [==============================] - 0s 107us/step - loss: 1.6733 - accuracy: 0.8882 - val_loss: 0.2250 - val_accuracy: 0.9720
Epoch 784/1000
331/331 [==============================] - 0s 89us/step - loss: 1.3901 - accuracy: 0.9094 - val_loss: 0.2242 - val_accuracy: 0.9720
Epoch 785/1000
331/331 [==============================] - 0s 72us/step - loss: 1.4821 - accuracy: 0.9003 - val_loss: 0.2247 - val_accuracy: 0.9720
Epoch 786/1000
331/331 [==============================] - 0s 90us/step - loss: 1.3971 - accuracy: 0.9063 - val_loss: 0.2242 - val_accuracy: 0.9720
Epoch 787/1000
331/331 [==============================] - 0s 84us/step - loss: 1.5875 - accuracy: 0.8912 - val_loss: 0.2243 - val_accuracy: 0.9720
Epoch 788/1000
331/331 [==============================] - 0s 73us/step - loss: 1.5686 - accuracy: 0.9003 - val_loss: 0.2241 - val_accuracy: 0.9720
Epoch 789/1000
331/331 [==============================] - 0s 107us/step - loss: 1.0227 - accuracy: 0.9305 - val_loss:

Epoch 839/1000
331/331 [==============================] - 0s 71us/step - loss: 1.0630 - accuracy: 0.9275 - val_loss: 0.2221 - val_accuracy: 0.9720
Epoch 840/1000
331/331 [==============================] - 0s 74us/step - loss: 1.3956 - accuracy: 0.9033 - val_loss: 0.2211 - val_accuracy: 0.9720
Epoch 841/1000
331/331 [==============================] - 0s 69us/step - loss: 2.0835 - accuracy: 0.8671 - val_loss: 0.2222 - val_accuracy: 0.9720
Epoch 842/1000
331/331 [==============================] - 0s 66us/step - loss: 2.0936 - accuracy: 0.8580 - val_loss: 0.2219 - val_accuracy: 0.9720
Epoch 843/1000
331/331 [==============================] - 0s 74us/step - loss: 1.8112 - accuracy: 0.8822 - val_loss: 0.2323 - val_accuracy: 0.9720
Epoch 844/1000
331/331 [==============================] - 0s 65us/step - loss: 2.0402 - accuracy: 0.8671 - val_loss: 0.2215 - val_accuracy: 0.9720
Epoch 845/1000
331/331 [==============================] - 0s 69us/step - loss: 1.5760 - accuracy: 0.8943 - val_loss: 0

Epoch 895/1000
331/331 [==============================] - 0s 69us/step - loss: 1.5330 - accuracy: 0.8973 - val_loss: 0.2228 - val_accuracy: 0.9720
Epoch 896/1000
331/331 [==============================] - 0s 79us/step - loss: 1.9418 - accuracy: 0.8731 - val_loss: 0.2193 - val_accuracy: 0.9720
Epoch 897/1000
331/331 [==============================] - 0s 69us/step - loss: 1.7082 - accuracy: 0.8882 - val_loss: 0.2184 - val_accuracy: 0.9720
Epoch 898/1000
331/331 [==============================] - 0s 69us/step - loss: 1.9022 - accuracy: 0.8761 - val_loss: 0.2188 - val_accuracy: 0.9720
Epoch 899/1000
331/331 [==============================] - 0s 80us/step - loss: 1.6252 - accuracy: 0.8912 - val_loss: 0.2189 - val_accuracy: 0.9720
Epoch 900/1000
331/331 [==============================] - 0s 69us/step - loss: 1.6694 - accuracy: 0.8912 - val_loss: 0.2189 - val_accuracy: 0.9720
Epoch 901/1000
331/331 [==============================] - 0s 77us/step - loss: 1.5730 - accuracy: 0.8973 - val_loss: 0

Epoch 951/1000
331/331 [==============================] - 0s 70us/step - loss: 1.4752 - accuracy: 0.9033 - val_loss: 0.2185 - val_accuracy: 0.9720
Epoch 952/1000
331/331 [==============================] - 0s 69us/step - loss: 1.9930 - accuracy: 0.8671 - val_loss: 0.2159 - val_accuracy: 0.9720
Epoch 953/1000
331/331 [==============================] - 0s 70us/step - loss: 1.3442 - accuracy: 0.9094 - val_loss: 0.2159 - val_accuracy: 0.9720
Epoch 954/1000
331/331 [==============================] - 0s 77us/step - loss: 1.3892 - accuracy: 0.9033 - val_loss: 0.2166 - val_accuracy: 0.9720
Epoch 955/1000
331/331 [==============================] - 0s 65us/step - loss: 1.6224 - accuracy: 0.8912 - val_loss: 0.2157 - val_accuracy: 0.9720
Epoch 956/1000
331/331 [==============================] - 0s 79us/step - loss: 1.5644 - accuracy: 0.9003 - val_loss: 0.2167 - val_accuracy: 0.9720
Epoch 957/1000
331/331 [==============================] - 0s 71us/step - loss: 2.2294 - accuracy: 0.8520 - val_loss: 0

In [36]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 43us/step
over-sampling test accuracy: 97.20%


In [37]:
############ Logistic Regression ############

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [39]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 97.58%


In [41]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 95.10%


In [43]:
############## Random Forest ##############

In [44]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [45]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.19%


In [46]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [47]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 95.10%


In [48]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [49]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.98275862 1.         0.94827586 1.         0.98214286]
0.9826354679802956


In [50]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98507463 0.96969697 0.98484848 0.98484848 0.93939394]
0.97277250113071
